In [6]:
QUERIES = ['pressure distributions']

In [8]:
from  sklearn.feature_extraction.text import CountVectorizer
from  sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import scipy.spatial.distance as ds
import warnings

In [9]:
warnings.filterwarnings("ignore")

def cosine_distance(vector_a: np.array, vector_b: np.array) -> float:
  return ds.cosine(vector_a, vector_b)

#Проверка, что функция работает правильно
assert cosine_distance(np.array([1, 0, 1, 1, 1]), np.array([0, 0, 1, 0, 0])) == 0.5

def jaccard_sim(vector_a: np.array, vector_b: np.array) -> float:
  vector_a = np.asarray(vector_a, np.bool)
  vector_b = np.asarray(vector_b, np.bool)
  return np.double(np.bitwise_and(vector_a, vector_b).sum()) / np.double(np.bitwise_or(vector_a, vector_b).sum())

#Проверка, что функция работает правильно
assert jaccard_sim(np.array([1, 0, 1, 0, 1]), np.array([0, 1, 1, 1, 1])) == 0.4

In [10]:
raw_query_data = [line.strip() for line in open("just.qry", "r").readlines()]
query_data = [""]

for query_part in raw_query_data:
  query_data[-1] += query_part + " "
  if query_part.endswith("."):
    query_data.append("")

encoder = CountVectorizer(binary=True)
encoded_data = encoder.fit_transform(query_data)
encoded_queries = encoder.transform(QUERIES)

id2term = {idx: term for term, idx in encoder.vocabulary_.items()}
non_zero_values_ids = encoded_data[0].nonzero()[1]

terms = [id2term[idx] for idx in non_zero_values_ids]

print("По мера Жаккара:")
for q_id, query in enumerate(encoded_queries):
  query = query.todense().A1
  docs = [doc.todense().A1 for doc in encoded_data]
  id2doc2similarity = [(doc_id, doc, jaccard_sim(query, doc)) for doc_id, doc in enumerate(docs)]
  closest = sorted(id2doc2similarity, key=lambda x: x[2], reverse=True)

  print("Q: %s:" %(QUERIES[q_id]))
  print("    %s\t%s" %("ID", "Коэффициент"))
  for closest_id, _, sim in closest[:2]:
    print("    %d\t%.2f" %(closest_id, sim))

print()

# Второе задание
tfidf_encoder = TfidfVectorizer()
tfidf_encoded_data = tfidf_encoder.fit_transform(query_data)
tfidf_encoded_queries = tfidf_encoder.transform(QUERIES)

print("Косинустное расстояние:")
for q_id, query in enumerate(tfidf_encoded_queries):

  # приводим к нужному типу
  query = query.todense().A1
  docs = [doc.todense().A1 for doc in tfidf_encoded_data]
  # Косинусное расстояние
  id2doc2similarity = [(doc_id, doc, cosine_distance(query, doc)) \
                       for doc_id, doc in enumerate(docs)]
  # сортируем по нему
  closest = sorted(id2doc2similarity, key=lambda x: x[2], reverse=False)

  print("Q: %s\nFOUND:" % QUERIES[q_id])

  for closest_id, _, sim in closest[:3]:
    print("    %d\t%.2f\t%s" %(closest_id, sim, query_data[closest_id]))

По мера Жаккара:
Q: pressure distributions:
    ID	Коэффициент
    204	0.29
    17	0.17

Косинустное расстояние:
Q: pressure distributions
FOUND:
    204	0.52	do viscous effects seriously modify pressure distributions . 
    17	0.58	are experimental pressure distributions on bodies of revolution at angle of attack available . 
    202	0.69	how accurate are existing analytical theories in estimating pressure distributions on cones at incidence,  at hypersonic speeds . 
